In [1]:
import rebound
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
import math # --- setup simulation ---

In [32]:
def update_plot(time):
    # Integrate simulation to the chosen time
    sim.integrate(time)
    op = rebound.OrbitPlot(sim)
    # Update existing plot
    op.update(updateLimits=False)
    op.particles.set_color(["green", "red"])

def random_point_on_orbit(r=1.0, z=0.0):
    """
    Returns a random point (x, y, z) on a circle of radius r_target in the plane z = plane_z.
    """
    theta = np.random.uniform(0, 2*np.pi)
    xt = r * np.cos(theta)
    yt = r * np.sin(theta)
    zt = z
    return (xt, yt, zt)

def IntruderVel(r0, fixed_speed, r_target=1.0, z=0.0):
    """
    Adds an intruder particle to the simulation `sim` starting at position r0 = (x0, y0, z0),
    and with initial velocity magnitude = fixed_speed, aiming toward a randomly selected
    target point on Earth’s orbit (radius ~ r_target, plane z = plane_z).
    """
    x0, y0, z0 = r0
    # pick random target point
    rt = random_point_on_orbit(r=r_target, z=z)
    xt, yt, zt = rt
    # compute direction vector toward that point
    d_vec = np.array([xt-x0, yt-y0, zt-z0], dtype=float)
    d_mag = np.linalg.norm(d_vec)
    if d_mag == 0:
        raise ValueError("Start position equals target position → direction undefined.")
    dir_unit = d_vec / d_mag
    
    # velocity vector with fixed magnitude
    v0_vec = dir_unit * fixed_speed
    return v0_vec

def InitializeOrbit(**kwargs):
    sim = rebound.Simulation()
    sim.units = ('yr', 'AU', 'Msun')
    sim.integrator = "ias15"

    # Add Sun
    sim.add(m=1.0, x=0., y=0., z=0., vx=0., vy=0., vz=0.)
    
    # Add Earth
    sim.add(m=3e-6, a=1.0, e=0.0167, inc=0.0, Omega=0.0, omega=0.0, f=0.0)
    sim.add(**kwargs)
    sim.move_to_hel()

    return sim
def UpdateOrbit(sim, time):
    sim.integrate(time)
    sim.move_to_hel()
    return sim

def distFromSun(earth):
    return np.sqrt(earth.x*earth.x+earth.y*earth.y+earth.z*earth.z)

intruder_mass = 1
r_start = 100
z = 0
rf = 2        # target radius ~ Earth’s orbit
intruder_speed = 12        # speed in AU/year (or whatever your sim uses)

r0 = random_point_on_orbit(r=r_start, z=z)
v = IntruderVel(r0, intruder_speed, r_target=rf, z=z)

 
    
sim = InitializeOrbit(m=intruder_mass,
            x=r0[0], y=r0[1], z=r0[2],
            vx=v[0], vy=v[1], vz=v[2])

# --- create interactive slider ---
interact(update_plot, time=FloatSlider(min=0, max=100, step=0.005, value=0, continuous_update=True))


interactive(children=(FloatSlider(value=0.0, description='time', step=0.005), Output()), _dom_classes=('widget…

<function __main__.update_plot(time)>

In [53]:
UpdateOrbit(sim,100)
orbit = sim.particles[1].orbit(primary=sim.particles[0])
print(orbit.a)

1.0077004275685952


In [39]:
import numpy as np
import csv
from tqdm import tqdm  # ✅ progress bar

intruder_mass = 1e-1
r_start = 30
z = 0
rf = 2           # target radius ~ Earth’s orbit
intruder_speed = 6.64         # speed in AU/year (or whatever your sim uses)
powersOfTen = 3
n = 10**powersOfTen 
# --- Write to CSV ---
with open('orbit_results.csv', mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['index','x', 'y', 'vx', 'vy', 'earth_a', 'ejected'])

    # ✅ Wrap the loop with tqdm
    for i in tqdm(range(1000), desc="Running simulations"):
        r0 = random_point_on_orbit(r=r_start, z=z)
        x0, y0, z0 = r0
        v_int = IntruderVel(r0, intruder_speed, r_target=rf, z=z)
        vx0, vy0, vz0 = v_int
        sim = InitializeOrbit(m=intruder_mass,
                    x=x0, y=y0, z=z0,
                    vx=vx0, vy=vy0, vz=vz0)
    
        sim = UpdateOrbit(sim, 100)
        earth = sim.particles[1].orbit(primary=sim.particles[0])
        if earth.a < 0:
            ejected = True
        else:
            ejected = False

        writer.writerow([i, x0, y0, vx0, vy0, earth.a, ejected])

print("\n✅ All simulations complete — results saved to orbit_results.csv")


Running simulations: 100%|██████████████████| 1000/1000 [00:30<00:00, 33.17it/s]


✅ All simulations complete — results saved to orbit_results.csv


In [47]:
from scipy.stats.qmc import Sobol

# --- define Sobol sampling parameters ---
d = 4  # dimensions: [mass, r_start, rf, intruder_speed]
powersOfTen = 4
n = 10**powersOfTen 

sampler = Sobol(d=d, scramble=True)
sobol_points = sampler.random(n)

# Define parameter ranges
param_min = np.array([1e-3,  30,   1,  0.1])   # [mass, r_start, rf, speed]
param_max = np.array([1,    100,  20,  15])

# Scale Sobol points to your ranges
samples = param_min + (param_max - param_min) * sobol_points

with open('orbit_results_sobol_'+str(powersOfTen)+'.csv', mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['index','mass','rTarget','x0', 'y0', 'vx0', 'vy0', 'earth_af', 'ejected'])
    # --- run simulation for each Sobol sample ---
    for i, (intruder_mass, r_start, rf, intruder_speed) in enumerate(
        tqdm(samples, desc="Running Sobol simulations", unit="sim")):
        
        z = 0
        r0 = random_point_on_orbit(r=r_start, z=z)
        v = IntruderVel(r0, intruder_speed, r_target=rf, z=z)
    
        sim = InitializeOrbit(m=intruder_mass, x=r0[0], y=r0[1], z=r0[2], vx=v[0], vy=v[1], vz=v[2])
    
        sim = UpdateOrbit(sim, 100)
        earth = sim.particles[1].orbit(primary=sim.particles[0])
        if earth.a < 0:
            ejected = True
        else:
            ejected = False

        writer.writerow([i,intruder_mass, rf, x0, y0, vx0, vy0, earth.a, ejected])
print("\n✅ All simulations complete — results saved to orbit_results_sobol_"+str(powersOfTen)+".csv")


Running Sobol simulations: 100%|█████████| 10000/10000 [04:59<00:00, 33.37sim/s]


✅ All simulations complete — results saved to orbit_results_sobol_4.csv
